In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading Required Data

data_train = pd.read_csv("/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv",encoding="utf-8", delimiter=',')
data_train.head()

## Bank Turnover Dataset
#### Can you predict if bank customers will turnover next cycle ?

In [ ]:
# importing the libraries
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 100)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_train.head()

In [ ]:
## Removing customerID and Row Number

data_train2 = data_train.drop(['RowNumber','CustomerId','Surname'],axis=1)
data_train2.shape

In [ ]:
data_train2.info()

In [ ]:
data_train2.head()

In [ ]:
data_train2.describe()

In [ ]:
## Lets findout the distribution of Churn: Yes and Nos First

# Good Practice: Always check if data set is balance or imbalance.
sns.set_style('whitegrid')
sns.countplot(x='Exited',data=data_train2,palette='RdBu_r')

#### Observations: 

- This is imbalanced dataset, There are different ways to cater it, But for the sake of learning deep learning, We can ignore it for now. 

In [ ]:
## There are some outliers in Balance and Estimated Salry. 1. Balance (min to 25%) is 0; while Estimated salary is 
## 11.58 rupees as minimum

In [ ]:
## Lets see this through dist plot

plt.hist(data_train2.EstimatedSalary, bins=5
         , rwidth=0.8)
plt.xlabel('EstimatedSalary')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.hist(data_train2.EstimatedSalary, bins=7
         , rwidth=0.8)
plt.xlabel('EstimatedSalary')
plt.ylabel('Count')
plt.yscale('log')
plt.show()

In [ ]:
# We will use percentiles technique to detect and remove outliers


MaxThershold = data_train2['EstimatedSalary'].quantile(0.999)
MaxThershold

In [ ]:
MinThershold = data_train2['EstimatedSalary'].quantile(0.015)
MinThershold

In [ ]:
data_train3 = data_train2[(data_train2.EstimatedSalary < MaxThershold) & (data_train2.EstimatedSalary > MinThershold)]
data_train3.head()

In [ ]:
data_train3.shape

In [ ]:
10000-9540

In [ ]:
## Detecting outlier in Balance

## Lets see this through dist plot

plt.hist(data_train3.Balance, bins=5
         , rwidth=0.8)
plt.xlabel('Balance')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.hist(data_train3.Balance, bins=7
         , rwidth=0.8)
plt.xlabel('Balance')
plt.ylabel('Count')
plt.yscale('log')
plt.show()

In [ ]:
# We will use percentiles technique to detect and remove outliers


MaxThershold = data_train3.Balance.quantile(0.999)
MaxThershold

In [ ]:
MinThershold = data_train3.Balance.quantile(0.370)
MinThershold

In [ ]:
data_train4 = data_train3[(data_train3.Balance < MaxThershold) & (data_train3.Balance > MinThershold)]
data_train4.head()

In [ ]:
data_train4.shape

In [ ]:
data_train4.dtypes

In [ ]:
#Lets seperate all as numerical/Categorical
# Findout Missing Value %age

statistics_of_data = []
for col in data_train4.columns:
  statistics_of_data.append((col,
                             data_train4[col].isnull().sum()*100/data_train4.shape[0],
                             data_train4[col].dtype
                             ))
stats_df = pd.DataFrame(statistics_of_data, columns=['Feature', 'missing_val', 'type'])

In [ ]:
stats_df.sort_values('missing_val', ascending=False)

In [ ]:
##No missing Values Found

In [ ]:
## Seperate out Numerical/Int Variables.
numerical_features = [feature for feature in data_train4.columns if data_train4[feature].dtypes != 'O' ]
print(len(numerical_features))
data_train4[numerical_features].head()

In [ ]:
## Numerical variables are usually of 2 type
## 1. Continous variable and Discrete Variables

discrete_feature = [feature for feature in numerical_features if len(data_train4[feature].unique())<25]
print(len(discrete_feature))
discrete_feature

In [ ]:
## Continous Features


continous_feature = [feature for feature in numerical_features if feature not in discrete_feature]
print("Continuous feature Count {}".format(len(continous_feature)))
data_train4[continous_feature].head()

In [ ]:
def print_unique_col_values(df):
    i=1
    for column in df:
        str = "{i}. {a} column have {b} unique values"
        print(str.format(i=i,a=column,b=df[column].unique()))
        i=i+1

In [ ]:
print_unique_col_values(data_train4[discrete_feature])

In [ ]:
# Now lets create visuals for comparison of continous/discrete and target vairable
##Exited: 0 -> No,1 ->Yes
tenure_churn_no = data_train4[data_train4.Exited==0].Tenure
tenure_churn_yes = data_train4[data_train4.Exited==1].Tenure

plt.xlabel("tenure")
plt.ylabel("Number Of Customers")
plt.title("Customer Churn Prediction Visualiztion")

plt.hist([tenure_churn_yes, tenure_churn_no], rwidth=0.95, color=['red','green'],label=['Churn=1','Churn=0'])
plt.legend()

In [ ]:
# Now lets create visuals for comparison of continous/discrete and target vairable
##Exited: 0 -> No,1 ->Yes
Age_churn_no = data_train4[data_train4.Exited==0].Age
Age_churn_yes = data_train4[data_train4.Exited==1].Age

plt.xlabel("Age")
plt.ylabel("Number Of Customers")
plt.title("Customer Churn Prediction Visualiztion")

plt.hist([Age_churn_yes, Age_churn_no], rwidth=0.95, color=['red','green'],label=['Churn=1','Churn=0'])
plt.legend()

In [ ]:
##Exited: 0 -> No,1 ->Yes
Balance_churn_no = data_train4[data_train4.Exited==0].Balance
Balance_churn_yes = data_train4[data_train4.Exited==1].Balance

plt.xlabel("Balance")
plt.ylabel("Number Of Customers")
plt.title("Customer Churn Prediction Visualiztion")

plt.hist([Balance_churn_yes, Balance_churn_no], rwidth=0.95, color=['red','green'],label=['Churn=1','Churn=0'])
plt.legend()

In [ ]:
#EstimatedSalary
##Exited: 0 -> No,1 ->Yes
EstimatedSalary_churn_no = data_train4[data_train4.Exited==0].EstimatedSalary
EstimatedSalary_churn_yes = data_train4[data_train4.Exited==1].EstimatedSalary

plt.xlabel("EstimatedSalary")
plt.ylabel("Number Of Customers")
plt.title("Customer Churn Prediction Visualiztion")

plt.hist([EstimatedSalary_churn_yes, EstimatedSalary_churn_no], rwidth=0.95, color=['red','green'],label=['Churn=1','Churn=0'])
plt.legend()

In [ ]:
#Now lets seperate our categorical features so that we can add other transformation uopn them
categoricalVariable = [feature for feature in data_train4.columns if data_train3[feature].dtype == 'O' ]
len(categoricalVariable)

In [ ]:
data_train4[categoricalVariable].head()

In [ ]:
#Male->1, Female 0
data_train4['Gender'].replace({'Female':1,'Male':0},inplace=True)

In [ ]:
#Lets apply One hot encoding for categorical column Geography

data_train5 = pd.get_dummies(data=data_train4, columns=['Geography'])
data_train5.columns

In [ ]:
data_train5.shape

In [ ]:
data_train5.sample(5)

In [ ]:
#Data Scaling of continous data

continous_feature

In [ ]:
### Data Scaling -->>Continous data only
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_train5[continous_feature] = scaler.fit_transform(data_train5[continous_feature])

In [ ]:
ij=1
for col in data_train5:
    str = "{ij}. {a} column have {b} unique values"
    print(str.format(ij = ij,a=col,b=data_train5[col].unique()))
    ij=ij+1

In [ ]:
### Train Test Split:::

X = data_train5.drop('Exited',axis='columns')
y = data_train5['Exited']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
len(X_train.columns)

### Now we are Builiding a Deep Learning Model (ANN) On keras/Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model_ChurnPred = keras.Sequential([
    keras.layers.Dense(12, input_shape=(12,), activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model_ChurnPred.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_ChurnPred.fit(X_train, y_train, epochs=200)

In [ ]:
## So we got 85% Accuracy, Now Save this model

# save model and architecture to single file
model_ChurnPred.save("model_ChurnPred.h5")
print("Saved model to disk")

In [ ]:
##Now Loading a Model
from keras.models import load_model
 
# load model
Model_Reloaded = load_model('model_ChurnPred.h5',compile=True)

In [ ]:
yp = Model_Reloaded.predict(X_test)
yp[:10]

In [ ]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
y_pred[:10]

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

In [ ]:
import seaborn as sn
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred)

plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
####################### ********** THE END ********************** ############################################